In [1]:
from __future__ import print_function
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words"""
  with zipfile.ZipFile(filename) as f:
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
  return data
  
words = read_data(filename)
print('Data size %d' % len(words))

Data size 17005207


In [10]:
import json

f = open('data/data.json','r')
sites = json.load(f)
f.close()

text = ''
for key in sites.keys():
    for article in sites[key]:
        text = text + article["content"]

         

In [31]:
str1 = text[0:10000]
str1

'Ryan James (left) and Mark Laughery (right) sit in their office space at the UW CoMotion Labs. The two started Pear Med last winter as part of their UW Software Entrepreneurship class and recently entered into negotiations with Seattle Children\'s Hospital to develop their virtual reality software for clinical use. (Photo by Timothy Kenney)\n A medical diagnosis can be one of the most terrifying and confusing days of a patient\'s life. Doctors often struggle to fully explain a person\'s illness to them because internal structures, like a tumor, can be hard to visualize. Soon, doctors could have a new tool that helps remove some of the confusion from a diagnosis: virtual and augmented reality. University of Washington startup Pear Med is taking advantage of that tech with VR and AR software that compiles medical scans into an interactive 3D model. The startup is currently negotiating a partnership with Seattle Children\'s Hospital to further develop their technology, which could revolu

In [35]:
str1.replace("\\","'")

'Ryan James (left) and Mark Laughery (right) sit in their office space at the UW CoMotion Labs. The two started Pear Med last winter as part of their UW Software Entrepreneurship class and recently entered into negotiations with Seattle Children\'s Hospital to develop their virtual reality software for clinical use. (Photo by Timothy Kenney)\n A medical diagnosis can be one of the most terrifying and confusing days of a patient\'s life. Doctors often struggle to fully explain a person\'s illness to them because internal structures, like a tumor, can be hard to visualize. Soon, doctors could have a new tool that helps remove some of the confusion from a diagnosis: virtual and augmented reality. University of Washington startup Pear Med is taking advantage of that tech with VR and AR software that compiles medical scans into an interactive 3D model. The startup is currently negotiating a partnership with Seattle Children\'s Hospital to further develop their technology, which could revolu